# 概览

与前面的线性回归相比，你会发现多类逻辑回归教程的结构跟其非常相似：**获取数据**、**定义模型**及**优化算法和求解**。

事实上，几乎所有的实际神经网络应用都有着同样结构。他们的主要区别在于模型的类型和数据的规模。每一两年会有一个新的优化算法出来，但它们基本都是随机梯度下降的变种。

通过本课学习到的技能：

- 多分类问题的损失函数与准确度度量函数，它们与回归问题中的差别定义
- Softmax的数值稳定问题
- 为什么trainer总是传入一个batchsize作为参数
- Gluon中使用GPU训练

## 获取数据

In [1]:
from mxnet import gluon
from mxnet import ndarray as nd

def transform(data, label):
    return data.astype('float32')/255, label.astype('float32')

mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform)
mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform)

In [2]:
data, label = mnist_train[0]
('example shape: ', data.shape, 'label:', label)

('example shape: ', (28, 28, 1), 'label:', 2.0)

In [3]:
batch_size = 256
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

In [4]:
for data, label in train_data:
    print(data.shape, label.shape)
    break

(256, 28, 28, 1) (256,)


## 定义模型

In [5]:
# 定义模型参数
num_inputs = 784
num_outputs = 10

W = nd.random_normal(shape=(num_inputs, num_outputs))
b = nd.random_normal(shape=(num_outputs,))

params=[W, b]

for param in params:
    param.attach_grad()  

In [6]:
def softmax(score):
    """
    input:
    - score N*D的矩阵，N代表样本的个数，D代表特征的长度
    output:
    - prob N*D的矩阵，每个元素代表了概率
    """
    max_value = nd.max(score, axis=1, keepdims=True)
    score = score - max_value
    exp_score = nd.exp(score)
    prob = exp_score / nd.sum(exp_score, axis=1, keepdims=True)
    return prob

In [7]:
# 定义model
def mlr(X):
    return softmax(nd.dot(X.reshape((X.shape[0], -1)), W) + b)

In [8]:
# 定义loss
def cross_entropy_loss(y, y_hat):
    return - nd.pick(nd.log(y), y_hat)

In [9]:
# 定义准确率计算函数
def accuracy(output, label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()

def evaluate_accuracy(data_iterator, net):
    acc = 0.
    for data, label in data_iterator:
        output = net(data)
        acc += accuracy(output, label)
    return acc / len(data_iterator)

In [10]:
evaluate_accuracy(test_data, mlr)

0.10048828125000001

In [11]:
# 定义优化算法
def SGD(params, lr):
    for param in params:
        param -= lr * param.grad

In [12]:
from mxnet import autograd

learning_rate = .1

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = mlr(data)
            loss = cross_entropy_loss(output, label)
        loss.backward()
        # 将梯度做平均，这样学习率会对batch size不那么敏感
        SGD(params, learning_rate / batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)

    test_acc = evaluate_accuracy(test_data, mlr)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))

Epoch 0. Loss: 3.700600, Train acc 0.438481, Test acc 0.569531
Epoch 1. Loss: 1.896220, Train acc 0.618440, Test acc 0.647754
Epoch 2. Loss: 1.573088, Train acc 0.667520, Test acc 0.681934
Epoch 3. Loss: 1.401253, Train acc 0.695728, Test acc 0.704492
Epoch 4. Loss: 1.289541, Train acc 0.713863, Test acc 0.716699


思考：为什么测试集的准确率比训练集的准确率高呢？

> 统计学习理论：基于独立同分布假设，训练误差的期望值不高于测试集误差的期望值。注意，这是期望值（单次实验结果可以是随机的）。当然，有很多实践方法（如正则化）可以使测试集准确率更大。

> 由于模型相对简单，并没有Overfiting的现象，所以模型有较大的bias error，这时测试集可能会有低的error

尝试增大学习率，你会发现结果马上回变成很糟糕，精度基本徘徊在随机的0.1左右。这是为什么呢？提示：

- 打印下output看看是不是有有什么异常
- 前面线性回归还好好的，这里我们在net()里加了什么呢？
- 如果给exp输入个很大的数会怎么样？
- 即使解决exp的问题，求出来的导数是不是还是不稳定？

请仔细想想再去对比下我们小伙伴之一@pluskid早年写的一篇[blog](http://freemind.pluskid.org/machine-learning/softmax-vs-softmax-loss-numerical-stability/)解释这个问题，看看你想的是不是不一样。

# Gluon 的版本， 使用GPU

In [13]:
def evaluate_accuracy(data_iterator, net):
    acc = 0.
    for data, label in data_iterator:
        output = net(data.as_in_context(mx.gpu()))
        acc += accuracy(output, label.as_in_context(mx.gpu()))
    return acc / len(data_iterator)

In [14]:
from mxnet import gluon
import mxnet as mx

# 定义模型
net = gluon.nn.Sequential()
net.add(gluon.nn.Flatten())
net.add(gluon.nn.Dense(10))
net.initialize(ctx=mx.gpu())

# 定义loss
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
# 优化器
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

# 训练
for epoch in range(10):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data.as_in_context(mx.gpu()))
            loss = softmax_cross_entropy(output, label.as_in_context(mx.gpu()))
        loss.backward()
        # trainer里总是传入batch size的原因是什么呢，为什么不直接在初始化trainer时写入呢。
        # 将梯度做平均，这样学习率会对batch size不那么敏感
        trainer.step(batch_size) 
        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label.as_in_context(mx.gpu()))

    test_acc = evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))

Epoch 0. Loss: 0.798806, Train acc 0.740481, Test acc 0.803418
Epoch 1. Loss: 0.576095, Train acc 0.809181, Test acc 0.817578
Epoch 2. Loss: 0.530283, Train acc 0.823582, Test acc 0.827148
Epoch 3. Loss: 0.506481, Train acc 0.829832, Test acc 0.835645
Epoch 4. Loss: 0.491218, Train acc 0.833876, Test acc 0.838672
Epoch 5. Loss: 0.477839, Train acc 0.838060, Test acc 0.837695
Epoch 6. Loss: 0.469924, Train acc 0.840409, Test acc 0.841895
Epoch 7. Loss: 0.462449, Train acc 0.843218, Test acc 0.845605
Epoch 8. Loss: 0.456732, Train acc 0.844963, Test acc 0.849316
Epoch 9. Loss: 0.451933, Train acc 0.845246, Test acc 0.848828


## 讨论Gluon版本的准确率更高的原因

1. net.initialize()：自带的初始化可能跟我们的随机初始化不完全一样，可能是更合适的。
2. softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss() 将softmax与交叉熵放在一起，数值更加稳定，也就降低了之前对softmax函数直接求导可能带来的风险。